<a href="https://colab.research.google.com/github/Arun-nexus/deep_learning/blob/main/pytorch_image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arun9x9","key":"bf041e53a52acba099f39cb5b826abc2"}'}

In [ ]:
import os
import zipfile

os.makedirs("/root/.kaggle", exist_ok=True)
!cp kaggle.json /root/.kaggle/
os.chmod("/root/.kaggle/kaggle.json", 600)

In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10

Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0


In [ ]:
with zipfile.ZipFile("animals10.zip", 'r') as zip_ref:
    zip_ref.extractall("animals10_dataset")

In [ ]:
train_path="animals10_dataset"

In [ ]:
train_path

In [ ]:
from torchvision import datasets, transforms,models

In [ ]:
train_transformer=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_test_transforms=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
dataset=datasets.ImageFolder(train_path,transform=train_transformer)

In [ ]:
train_size=int(0.7*len(dataset))
val_size=int(0.15*len(dataset))
test_size=len(dataset)-train_size-val_size

In [ ]:
from torch.utils.data import random_split,DataLoader

In [ ]:
train_ds,val_ds,test_ds=random_split(dataset,[train_size,val_size,test_size])

In [ ]:
train_loader=DataLoader(train_ds,batch_size=32,shuffle=True)
val_loader=DataLoader(val_ds,batch_size=32,shuffle=False)
test_loader=DataLoader(test_ds,batch_size=32,shuffle=False)

In [ ]:
import torch.nn as nn

In [ ]:
class AnimalCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128*16*16,512), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [ ]:
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AnimalCNN().to(device)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
best_acc = 0.0
for epoch in range(1,21):
    model.train()
    train_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)

    model.eval()
    val_loss=0.0; correct=0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()*imgs.size(0)
            pred = outputs.argmax(dim=1)
            correct += pred.eq(labels).sum().item()

    val_acc = correct / len(val_ds)
    print(f"Epoch {epoch:>2} | Train Loss: {train_loss/len(train_ds):.4f} | Val Loss: {val_loss/len(val_ds):.4f} | Val Acc: {val_acc:.3f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_animals10_cnn.pth')

print(f"🏆 Best Validation Accuracy: {best_acc:.3f}")

Epoch  1 | Train Loss: 0.0040 | Val Loss: 0.0000 | Val Acc: 1.000
Epoch  2 | Train Loss: 0.0000 | Val Loss: 0.0000 | Val Acc: 1.000
